In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv', parse_dates=['date'], infer_datetime_format=True)
test = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv', parse_dates=['date'], infer_datetime_format=True)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import sys
import numpy as np # linear algebra
import os
import holidays


def createFeatures(df_in,offset,labelwith):
	us_holidays = holidays.UnitedStates()
	df = df_in.copy()

	df['dayofweek'] = df['date'].dt.dayofweek
	df['quarter'] = df['date'].dt.quarter
	df['month'] = df['date'].dt.month
	df['year'] = df['date'].dt.year
	df['dayofyear'] = df['date'].dt.dayofyear
	df['dayofmonth'] = df['date'].dt.day
	df['weekofyear'] = df['date'].map(lambda x: x.weekofyear)


	df['holiday'] =  df['date'].map(lambda x: int(x in us_holidays))

	window = df['sales'].rolling(window=offset)
	df['mean'] = window.mean()
	df['min'] = window.min()
	df['max'] = window.max()

	df = df[offset-1:]

	df['mean'] = df['mean'].shift(periods=labelwith)
	df['min'] = df['min'].shift(periods=labelwith)
	df['max'] = df['max'].shift(periods=labelwith)

	df = df[labelwith:]


	return df




In [ ]:

def perform(train,test,datesplit):
	global previous_days_info,labelwith,featureSet

	train_original = train.copy()

	train = createFeatures(train,offset=previous_days_info,labelwith=labelwith)
	scaler = MinMaxScaler()
	train['sales'] = scaler.fit_transform(train['sales'].values.reshape(-1, 1))

	# Create a Random Forest object
	rf = RandomForestRegressor()

	# Train a model
	rf.fit(X=train[featureSet], y=train['sales'])

	# prepare test data
	df = pd.concat([train_original,test],axis=0, sort=True)

	test = createFeatures(df,offset= previous_days_info,labelwith=labelwith)
	test = test[-labelwith:] # get the test dataset

	# Get predictions for the test set
	test['sales'] = rf.predict(test[featureSet])

	readdata = scaler.inverse_transform([test['sales']])[0]
	readdata = [round(x) for x in readdata]
	test['sales'] = readdata

	# return test[['date','item', 'sales']]
	return test[['sales']]


In [ ]:
import pandas as pd
import sys
import numpy as np # linear algebra
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import os

labelwith = 90
previous_days_info = 90
featureSet = ['dayofweek','quarter','month','year','dayofyear','dayofmonth','weekofyear','min','max','mean','holiday']


us_holidays = holidays.UnitedStates()


storeids = train['store'].unique()
itemids = train['item'].unique()


ans = pd.DataFrame()

for item in itemids:
    iteminfo = train.loc[train['item']==item]
    storeids = iteminfo['store'].unique()
    for sid in storeids:
        one_train = iteminfo[iteminfo['store'] == sid]
        one_test = test[(test['store'] == sid) & (test['item'] == item)]
        # titems = titems.drop("id",axis=1)
        if len(one_test) == 0:
            continue

        rlt = perform(one_train,one_test,'2018-01-01')
        ans = pd.concat([ans,rlt],axis=0, sort=True) 

ans.to_csv('submission.csv',index_label='id')
